## Elo Rating Tuning method

This code helps with the tuning of the K value of the Elo method.  

We also produce the probalitity mean square error via: 

http://opisthokonta.net/?p=1387

Reference: https://www.gautamnarula.com/rating/

In [12]:
import math 
  
# Function to calculate the Probability 
def Probability(rating1, rating2): 
  
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * (rating1 - rating2) / 400)) 
  
# Function to calculate Elo rating 
# K is a constant. 
# Player A wins over Player B.  
# tie = true if tie, false otherwise
def EloRating(Ra, Rb, K, tie): 
    
    # To calculate the Winning 
    # Probability of Player B 
    Pb = Probability(Ra, Rb) 
  
    # To calculate the Winning 
    # Probability of Player A 
    Pa = Probability(Rb, Ra) 
  
    # Updating the Elo Ratings 
    if tie:
       Ra = Ra + K * (1/2 - Pa) 
       Rb = Rb + K * (1/2 - Pb) 
    else:        
       Ra = Ra + K * (1 - Pa) 
       Rb = Rb + K * (0 - Pb) 
    
    return Ra, Rb

In [13]:
gameFilename = '2014games.txt'
teamFilename = '2014teams.txt'

### Load the team names into an array

In [14]:
import pandas as pd

teamNames = pd.read_csv(teamFilename, header = None)
numTeams = len(teamNames)

### Load the games

In [15]:
# columns of games are:
#	column 0 = days since 1/1/0000
#	column 1 = date in YYYYMMDD format
#	column 2 = team1 index
#	column 3 = team1 homefield (1 = home, -1 = away, 0 = neutral)
#	column 4 = team1 score
#	column 5 = team2 index
#	column 6 = team2 homefield (1 = home, -1 = away, 0 = neutral)
#	column 7 = team2 score
games = pd.read_csv(gameFilename, header = None)
numGames = len(games)

### Create Elo ratings

In [16]:
import numpy as np

eloRatings = np.zeros(numTeams)

numberTeamsToPrint = 20

for i in range(numGames):
    team1ID = games.loc[i, 2] - 1 # subtracting 1 since python indexes at 0
    team1Score = games.loc[i, 4]
    team2ID = games.loc[i, 5] - 1 # subtracting 1 since python indexes at 0
    team2Score = games.loc[i, 7]

    if team1Score > team2Score:
        team1Rating, team2Rating = EloRating(eloRatings[team1ID], eloRatings[team2ID], 32, False)
    elif team1Score < team2Score:
        team2Rating, team1Rating = EloRating(eloRatings[team2ID], eloRatings[team1ID], 32, False)
    else:  
        team1Rating, team2Rating = EloRating(eloRatings[team1ID], eloRatings[team2ID], 32, True)
        
    eloRatings[team1ID] = team1Rating
    eloRatings[team2ID] = team2Rating

### Sort and print the ranking of teams

In [17]:
iSort = np.argsort(-eloRatings)

print('\n\n************** ELO Rating Method **************\n')
print('===========================')
print('Rank   Rating      Team   ')
print('===========================')
for i in range(numberTeamsToPrint):
    print(f'{i+1:4d}   {eloRatings[iSort[i]]:.5f}  {teamNames.loc[iSort[i],1]}')

print('')   # extra carriage return



************** ELO Rating Method **************

Rank   Rating      Team   
   1   406.24084   Wichita_St
   2   401.31865   Florida
   3   333.08314   Arizona
   4   328.33594   SF_Austin
   5   317.45380   Louisville
   6   311.20976   Villanova
   7   305.44670   Virginia
   8   296.40762   San_Diego_St
   9   282.65796   New_Mexico
  10   272.45027   Syracuse
  11   267.79992   Gonzaga
  12   265.71599   Cincinnati
  13   259.62314   Iowa_St
  14   259.20766   Harvard
  15   252.17122   Michigan
  16   249.00610   Duke
  17   248.28522   Wisconsin
  18   245.58657   Creighton
  19   244.86308   UCLA
  20   244.22964   NC_Central



### Calculate predictability of the method

In [18]:
numberCorrectPredictions = 0
for i in range(numGames):
    team1ID = games.loc[i, 2] - 1 
    team1Score = games.loc[i, 4]
    team2ID = games.loc[i, 5] - 1 
    team2Score = games.loc[i, 7]
    
    if team1Score > team2Score and eloRatings[team1ID] > eloRatings[team2ID]:
        numberCorrectPredictions += 1
    elif team2Score > team1Score and eloRatings[team2ID] > eloRatings[team1ID]:
        numberCorrectPredictions += 1
    elif team1Score == team2Score and eloRatings[team1ID] == eloRatings[team2ID]:
        numberCorrectPredictions += 1

print(f'Predictability: {numberCorrectPredictions/numGames*100:.2f}%') 


Predictability: 74.84%


### Calculate mean square error

In [22]:
meanSquareError = 0
for i in range(numGames):
    team1ID = games.loc[i, 2] - 1 
    team1Score = games.loc[i, 4]
    team2ID = games.loc[i, 5] - 1 
    team2Score = games.loc[i, 7]

    # To calculate the Winning 
    # Probability of team 1
    P1 = Probability(eloRatings[team1ID], eloRatings[team2ID]) 
    # To calculate the Winning 
    # Probability of team 2
    P2 = Probability(eloRatings[team2ID], eloRatings[team1ID])     
    
    if team1Score > team2Score:
        meanSquareError += (P1 - 1)**2   + (P2 - 0)**2
    elif team2Score > team1Score:
        meanSquareError += (P1 - 0)**2   + (P2 - 2)**2
    elif team1Score == team2Score and eloRatings[team1ID] == eloRatings[team2ID]:
        meanSquareError += (P1 - 0.5)**2 + (P2 - 0.5)**2

print('Mean Square Error = %f' % (meanSquareError/numGames) )


Mean Square Error = 0.646084
